In [1]:
import math
import torch
import gpytorch
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

In [2]:
import numpy as np

### Set up training data

In [3]:
# We will use the simplest form of GP model, exact inference
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, ndims):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        
        self.mean_module = gpytorch.means.ConstantMean()
        self.base_covar_module = gpytorch.kernels.RBFKernel()
        self.covar_module = gpytorch.kernels.ProductStructureKernel(
            gpytorch.kernels.ScaleKernel(self.base_covar_module), num_dims=ndims
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

class BayesianFitter(object):
    def __init__(self, train_x, train_y):
        self.train_x = torch.from_numpy(train_x.astype(np.float32))
        self.train_y = torch.from_numpy(train_y.astype(np.float32))
        
        # initialize likelihood and model
        self.likelihood = gpytorch.likelihoods.GaussianLikelihood()
        self.model = ExactGPModel(self.train_x, self.train_y, self.likelihood, train_x.shape[1])

        self.maxCGiterations = 50
        
    def findHyperparameters(self):
        # Find optimal model hyperparameters
        self.model.train()
        self.likelihood.train()

        # Use the adam optimizer
        optimizer = torch.optim.Adam([
            {'params': self.model.parameters()},  # Includes GaussianLikelihood parameters
        ], lr=0.1)

        # "Loss" for GPs - the marginal log likelihood
        mll = gpytorch.mlls.ExactMarginalLogLikelihood(self.likelihood, self.model)

        for i in range(self.maxCGiterations):
            # Zero gradients from previous iteration
            optimizer.zero_grad()
            # Output from model
            output = self.model(self.train_x)
            # Calc loss and backprop gradients
            loss = -mll(output, self.train_y)
            loss.backward()
            optimizer.step()
        # Get into evaluation (predictive posterior) mode
        self.model.eval()
        self.likelihood.eval()    
        
    def predict_f(self, test_x):
        test_x = torch.from_numpy(test_x.astype(np.float32))
        with gpytorch.settings.fast_pred_var(), torch.no_grad(), gpytorch.settings.max_root_decomposition_size(25):
            observed_pred = self.likelihood(self.model(test_x))
            return (observed_pred.mean - 2*observed_pred.stddev).numpy()

    def predict_fp(self, test_x):
        test_x = torch.from_numpy(test_x.astype(np.float32))
        test_x.requires_grad = True
        with gpytorch.settings.fast_pred_var(), gpytorch.settings.max_root_decomposition_size(25):
            observed_pred = self.likelihood(self.model(test_x))
            return torch.autograd.grad((observed_pred.mean - 2*observed_pred.stddev).sum(), test_x)[0].numpy()

In [4]:
def real_f(x):
    return np.sin(x*2*math.pi).sum(1)

In [5]:
train_x = np.random.random(size=(500,2))*4
test_x =  np.random.random(size=(1000,2))*4
train_y = real_f(train_x) + np.random.normal(size=train_x.shape[0], scale=0.04) 
test_y = real_f(test_x)

In [6]:
train_x.shape, train_y.shape

((500, 2), (500,))

In [7]:
test_x.shape, test_y.shape

((1000, 2), (1000,))

In [8]:
model = BayesianFitter(train_x, train_y)

In [9]:
model.findHyperparameters()

In [10]:
pred = model.predict_f(test_x)

In [11]:
pred_p = model.predict_fp(test_x)

In [12]:
pred_p

array([[-0.29454672,  2.7328866 ],
       [ 3.2426677 , -1.2749121 ],
       [ 1.9540269 , -6.193016  ],
       ...,
       [-1.155921  ,  6.17285   ],
       [ 0.5903637 , -0.12933743],
       [-1.2933826 ,  5.688449  ]], dtype=float32)

## Plot the Training Surface

In [13]:
%matplotlib widget

In [14]:
x, y, z = train_x[:,0], train_x[:,1], train_y

In [15]:
fig = plt.figure()
ax = Axes3D(fig)
surf = ax.plot_trisurf(x, y, z, cmap=cm.jet, linewidth=0.1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
z = pred = model.predict_f(test_x)

In [18]:
fig = plt.figure()
ax = Axes3D(fig)
surf = ax.plot_trisurf(x, y, z, cmap=cm.jet, linewidth=0.1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Plot the Testing Surface

In [19]:
x, y, z = test_x[:,0], test_x[:,1], test_y

In [20]:
fig = plt.figure()
ax = Axes3D(fig)
surf = ax.plot_trisurf(x, y, z, cmap=cm.jet, linewidth=0.1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
z = pred

In [23]:
fig = plt.figure()
ax = Axes3D(fig)
surf = ax.plot_trisurf(x, y, z, cmap=cm.jet, linewidth=0.1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
z = pred_p[:,0]

In [25]:
fig = plt.figure()
ax = Axes3D(fig)
surf = ax.plot_trisurf(x, y, z, cmap=cm.jet, linewidth=0.1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
z = pred_p[:,1]

In [27]:
fig = plt.figure()
ax = Axes3D(fig)
surf = ax.plot_trisurf(x, y, z, cmap=cm.jet, linewidth=0.1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …